In [ ]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from PIL import Image
from scipy.stats import gaussian_kde

import warnings

# FutureWarning 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# PUGB API KEY

In [ ]:
# PUBG API URL 및 API KEY 설정
API_URL = "https://api.pubg.com"
#API_KEY = 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9

In [33]:
# 요청 헤더 설정
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/vnd.api+json"
}

# player id 별 matchid
def get_player_id(player_name):
    endpoint = f"/shards/steam/players?filter[playerNames]={player_name}"
    response = requests.get(API_URL + endpoint, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data # data['data'][0]['relationships']['matches']['data']
        else:
            print("플레이어를 찾을 수 없습니다.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None



# matchid 별 경기
def get_match_details(match_id):
    url = 'https://api.pubg.com/shards/steam/matches/{}'
    url = url.format(match_id)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None



* 7일동안의 매치데이터

In [106]:

# PUBG API endpoint and headers
api_url = "https://api.pubg.com/shards/steam/matches"

# Calculate the start and end times for the past 7 days in UTC
end_time = datetime.utcnow()
start_time = end_time - timedelta(days=7)

# Format the times in ISO 8601 format
end_time_str = '2024-06-03T00:00:00Z' #end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
start_time_str = '2024-05-30T00:00:00Z' #start_time.strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL with the query parameters
params = {
    "filter[createdAt-start]": start_time_str,
    "filter[createdAt-end]": end_time_str
}

# Make the API request
response = requests.get(api_url, headers=headers, params=params)
data = response.json()


In [107]:
data

{'code': 'PAGE_NOT_FOUND', 'message': 'Page not found'}

In [103]:
m = data['data']['relationships']['matches']['data']
match_7day = [match['id'] for match in m]
len(match_7day)

960

* 특정 유저별 최근 matchid 가져오기

In [ ]:
player_name = 'VINGMINGVINGMING' # xiaosusu-9527 # 5b3f0f4b-7f8b-4d61-88a6-715a7186b7fa
data = get_player_id(player_name)
match_ids = data['data'][0]['relationships']['matches']['data']
match_ids = [match['id'] for match in match_ids]


In [104]:
match_id =  'af3e06f5-d61b-4eac-b914-97c27bc23cda' # '8dbbf996-cbc2-49a1-bb41-dc84144b4fa1'  #'5b3f0f4b-7f8b-4d61-88a6-715a7186b7fa'
match_data = get_match_details(match_7day[1])

* includid 매치별 참가자 기본통계
* 전투로그 같은 세부 정보는 asset key 를 이용해서 수집해야함

In [ ]:
match_data['included']

### 매치 기본 통계와 Detail Log

In [ ]:
match_static = pd.json_normalize(match_data['included'])
player_match_static = match_static[match_static['attributes.stats.name'].notnull()]
vingming = match_static[match_static['attributes.stats.name'] == player_name]


In [62]:
# json_normalize 사용
df = pd.json_normalize(
    match_data['included'],
    sep='_'
)

# 열 이름을 원하는 형식으로 수정
df.columns = df.columns.str.replace('attributes_stats_', '').str.replace('attributes_', '')


In [64]:
# 매치 기본통계정보 는 icluded 키 안에
included = match_data['included']

# 세부로그는 asset key 안의 telemetry 주소로 다시 호출해야 한다.
telemetry_URL = [item for item in match_data['included'] if item['type'] == 'asset'][0]['attributes']['URL']
telemetry_URL

'https://telemetry-cdn.pubg.com/bluehole-pubg/steam/2024/05/24/15/03/d67b2d54-19de-11ef-88ce-c2a569f64902-telemetry.json'

In [ ]:
response2 = requests.get(telemetry_URL, headers=headers)
log = response2.json()

In [ ]:
# frame_df = pd.DataFrame(log)
# df = pd.DataFrame(frame_df['info']['participants'])
logs = log[1:]
df = pd.json_normalize(logs)


# 주제 : 총기 밸런스

총기 밸런스 패치를 하기위해 어떤 정보를 봐야할까? 그리고 어떻게 전처리해야할까


라운드별 총기 인기도, 총기 상성(예를 들어 밴달 사용자와 팬텀 사용자의 맞대결에서 누가 우위에 있는지)


## 조건

* 다른 사람 개입 없이 1:1 싸움인 경우 (혹은 제3자 개입이 있는 경우 같이)
* 미러전은 따로 봐야함 (스카 vs 스카)
* 일방적인 공격이 아니어야함 (전면전 이어야함)
* 거리별로 다르게 봐야함
* 파츠별로 다르게 봐야함

## 분석지표 정리

* 정확도, 탄퍼짐, 반동회복, 발사한 탄환수, 적중한 탄환

발로란트에서는 5초 이내에 발생한 멀티킬을 총기의 난사 제어를 알아보는데 지표로 사용

* 부위별 적중도
* 거리별 적중도 (5m 단위?)
* 총기 인기도
* 총기별 게임승률
* 1:1 전면전 승률
*

# 29.1 패치노트

SCAR-L과 AKM의 경우, 낮은 사용성과 승률로 인해 상향이 필요하다는 의견
* AKM : 피해량을 소폭 증가 (다른 AR에 비해 피해량이 강화)
* SCAR-L : 낮은 연사 속도를 소폭 증가 + 수직/수평 반동 각 4%씩 감소

* 판처파우스트에 장착 모션을 추가. 빠르게 줍거나 교체하여 발사할 수 없도록 조정

* Mk12의 경우 현재 가장 범용적으로 사용되는 DMR 총기 -> 사운드 인지 어려움(소음기+라이트그립 장착시 OP) -> 사운드만 조정




In [ ]:
# df.sort_values(by=['_D'],ascending=True).head(1000)
# dff = df[~df['_T'].isin(['LogPlayerCreate', 'LogPlayerLogin'])].sort_values(by=['character.name']).head(800)
import re
kill_df = df[df['_T'].isin(['LogPlayerKillV2','LogPlayerAttack','LogPlayerTakeDamage'])] # 'LogPlayerKillV2','LogPlayerAttack',
kill_df = kill_df.filter(regex=re.compile('_D|_T|attack|victim|dBNO|finish'))
kill_df[(kill_df['attacker.name']== 'DarkBEEER') | (kill_df['victim.name'] == 'DarkBEEER') | (kill_df['dBNOMaker.name'] == 'DarkBEEER') | (kill_df['finisher.name'] == 'DarkBEEER')].sort_values(by=['_D'])


,_D,_T,attackId,attackType,attacker.name,attacker.teamId,attacker.health,attacker.location.x,attacker.location.y,attacker.location.z,attacker.ranking,attacker.individualRanking,attacker.accountId,attacker.isInBlueZone,attacker.isInRedZone,attacker.zone,attacker.type,victim.name,victim.teamId,victim.health,victim.location.x,victim.location.y,victim.location.z,victim.ranking,victim.individualRanking,victim.accountId,victim.isInBlueZone,victim.isInRedZone,victim.zone,victim.type,victimWeapon,victimWeaponAdditionalInfo,dBNOId,victimGameResult.rank,victimGameResult.gameResult,victimGameResult.teamId,victimGameResult.stats.killCount,victimGameResult.stats.distanceOnFoot,victimGameResult.stats.distanceOnSwim,victimGameResult.stats.distanceOnVehicle,victimGameResult.stats.distanceOnParachute,victimGameResult.stats.distanceOnFreefall,victimGameResult.stats.bpRewardDetail.byPlayTime,victimGameResult.stats.bpRewardDetail.byRanking,victimGameResult.stats.bpRewardDetail.byKills,victimGameResult.stats.bpRewardDetail.byDamageDealt,victimGameResult.stats.bpRewardDetail.boostAmount,victimGameResult.stats.bpRewardDetail.byModeScore,victimGameResult.stats.arcadeRewardDetail.byPlayTime,victimGameResult.stats.statTrakDataPairs,victimGameResult.stats.headshotStatTrakDataPairs,victimGameResult.accountId,victimGameResult.isRewardAbuse,dBNOMaker.name,dBNOMaker.teamId,dBNOMaker.health,dBNOMaker.location.x,dBNOMaker.location.y,dBNOMaker.location.z,dBNOMaker.ranking,dBNOMaker.individualRanking,dBNOMaker.accountId,dBNOMaker.isInBlueZone,dBNOMaker.isInRedZone,dBNOMaker.zone,dBNOMaker.type,dBNODamageInfo.damageReason,dBNODamageInfo.damageTypeCategory,dBNODamageInfo.damageCauserName,dBNODamageInfo.additionalInfo,dBNODamageInfo.distance,dBNODamageInfo.isThroughPenetrableWall,finisher.name,finisher.teamId,finisher.health,finisher.location.x,finisher.location.y,finisher.location.z,finisher.ranking,finisher.individualRanking,finisher.accountId,finisher.isInBlueZone,finisher.isInRedZone,finisher.zone,finisher.type,finishDamageInfo.damageReason,finishDamageInfo.damageTypeCategory,finishDamageInfo.damageCauserName,finishDamageInfo.additionalInfo,finishDamageInfo.distance,finishDamageInfo.isThroughPenetrableWall,dBNOMaker,attacker,finisher
156,2024-05-28T16:18:53.293Z,LogPlayerAttack,64.0,Weapon,DarkBEEER,7.0,100.000000,125861.226562,703949.81250,10844.690430,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,2024-05-28T16:18:55.031Z,LogPlayerAttack,65.0,Weapon,DarkBEEER,7.0,100.000000,125501.226562,703709.81250,10837.120117,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14247,2024-05-28T16:28:15.865Z,LogPlayerAttack,16778093.0,Weapon,DarkBEEER,7.0,100.000000,365549.687500,437045.96875,4798.779785,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14249,2024-05-28T16:28:15.931Z,LogPlayerAttack,16778094.0,Weapon,DarkBEEER,7.0,100.000000,365549.687500,437045.96875,4798.779785,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

* LogPlayerTakeDamage 에서 attacker.name	가 없는 경우 자기장에 맞은 것
* 사람간의 전투를 봐야하므로 _T 가 LogPlayerTakeDamage 이고 attacker.name 이 nan 인경우 제거

In [ ]:
kill_df[kill_df['attacker.name'].notnull()].head(10)

,_D,_T,attackId,attackType,attacker.name,attacker.teamId,attacker.health,attacker.location.x,attacker.location.y,attacker.location.z,attacker.ranking,attacker.individualRanking,attacker.accountId,attacker.isInBlueZone,attacker.isInRedZone,attacker.zone,attacker.type,victim.name,victim.teamId,victim.health,victim.location.x,victim.location.y,victim.location.z,victim.ranking,victim.individualRanking,victim.accountId,victim.isInBlueZone,victim.isInRedZone,victim.zone,victim.type,victimWeapon,victimWeaponAdditionalInfo,dBNOId,victimGameResult.rank,victimGameResult.gameResult,victimGameResult.teamId,victimGameResult.stats.killCount,victimGameResult.stats.distanceOnFoot,victimGameResult.stats.distanceOnSwim,victimGameResult.stats.distanceOnVehicle,victimGameResult.stats.distanceOnParachute,victimGameResult.stats.distanceOnFreefall,victimGameResult.stats.bpRewardDetail.byPlayTime,victimGameResult.stats.bpRewardDetail.byRanking,victimGameResult.stats.bpRewardDetail.byKills,victimGameResult.stats.bpRewardDetail.byDamageDealt,victimGameResult.stats.bpRewardDetail.boostAmount,victimGameResult.stats.bpRewardDetail.byModeScore,victimGameResult.stats.arcadeRewardDetail.byPlayTime,victimGameResult.stats.statTrakDataPairs,victimGameResult.stats.headshotStatTrakDataPairs,victimGameResult.accountId,victimGameResult.isRewardAbuse,dBNOMaker.name,dBNOMaker.teamId,dBNOMaker.health,dBNOMaker.location.x,dBNOMaker.location.y,dBNOMaker.location.z,dBNOMaker.ranking,dBNOMaker.individualRanking,dBNOMaker.accountId,dBNOMaker.isInBlueZone,dBNOMaker.isInRedZone,dBNOMaker.zone,dBNOMaker.type,dBNODamageInfo.damageReason,dBNODamageInfo.damageTypeCategory,dBNODamageInfo.damageCauserName,dBNODamageInfo.additionalInfo,dBNODamageInfo.distance,dBNODamageInfo.isThroughPenetrableWall,finisher.name,finisher.teamId,finisher.health,finisher.location.x,finisher.location.y,finisher.location.z,finisher.ranking,finisher.individualRanking,finisher.accountId,finisher.isInBlueZone,finisher.isInRedZone,finisher.zone,finisher.type,finishDamageInfo.damageReason,finishDamageInfo.damageTypeCategory,finishDamageInfo.damageCauserName,finishDamageInfo.additionalInfo,finishDamageInfo.distance,finishDamageInfo.isThroughPenetrableWall,dBNOMaker,attacker,finisher
156,2024-05-28T16:18:53.293Z,LogPlayerAttack,64.0,Weapon,DarkBEEER,7.0,100.0,125861.226562,703949.8125,10844.690430,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,2024-05-28T16:18:55.031Z,LogPlayerAttack,65.0,Weapon,DarkBEEER,7.0,100.0,125501.226562,703709.8125,10837.120117,0.0,0.0,account.86de66b31cfa4e6592f030b95e2cf781,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2024-05-28T16:18:56.276Z,LogPlayerAttack,66.0,Weapon,Godlaijungle,3.0,100.0,123062.468750,702913.2500,10850.099609,0.0,0.0,account.0f25e453ac894e1694b39ddd2686782e,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,2024-05-28T16:18:56.944Z,LogPlayerAttack,67.0,Weapon,Godlaijungle,3.0,100.0,123056.726562,703035.0625,10758.129883,0.0,0.0,account.0f25e453ac894e1694b39ddd2686782e,False,False,[],user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
kill_df['victimWeapon'].value_counts()